<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-13 10:33:05
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.41 C
-------------------
Today PnL: 16.85 K (0.12%)
Current PnL: -28.11 L (-17.93%)
CY Booked + Current PnL: -13.82 L (-8.81%)
-------------------
Total profit:  1.72 L
Total loss:  -29.83 L
-------------------
Total Booked + Current PnL: 13.16 L (10.14%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.14%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 93.57 L (66.38%)
Deployed:  1.30 C
Current:  1.41 C
CAGR/XIRR %: 4.46%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,0.27,16.02,4.64,21.40,14289.0,42518.0,307944.0,7.60,65.0,X-LC,6.13,14.0,2.98,2.18,23.36,XY25,NTT,FMCG
17,COALINDIA,484.83,-0.23,6.06,12.41,19.23,19804.0,9120.0,159581.0,24.27,71.0,L-LC,8.34,182.0,0.46,1.13,28.63,XY25,ATH,MINING
77,TTKPRESTIG,770.00,0.41,-22.01,28.33,0.09,22267.0,-22177.0,78600.0,82.50,36.0,M-SC,3.45,253.0,-1.00,0.56,3.26,OX40N,NTT,DURABLES
49,MASFIN,398.61,-0.05,-3.98,27.11,22.05,25505.0,-3900.0,94080.0,-17.34,47.0,H-SC,5.72,168.0,-0.15,0.67,36.75,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,0.02,-14.91,17.55,0.02,26066.0,-26032.0,148527.0,126.56,52.0,M-SC,8.57,240.0,-1.00,1.05,29.19,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ALKYLAMINE,4546.37,3.01,-32.17,185.47,93.63,127014.0,-32481.0,68482.0,-27.39,48.0,H-SC,16.79,143.0,-0.26,0.49,3.53,SR,ATH,CHEMICALS
26,GILLETTE,11206.78,2.46,-5.07,39.48,32.42,95161.0,-12861.0,241035.0,-21.96,45.0,X-SC,12.33,87.0,-0.14,1.71,9.27,X40,ATH,FMCG
66,SIS,528.00,2.43,-23.53,58.58,21.26,49736.0,-26130.0,84902.0,2003.63,50.0,H-SC,4.72,163.0,-0.53,0.60,14.71,OX40N,NTT,MISC
55,RAJESHEXPO,518.00,2.14,-67.78,210.55,0.07,93067.0,-92975.0,44202.0,1519.95,35.0,L-SC,1.47,269.0,-1.00,0.31,9.93,OX40N,NTT,JEWELLERY
13,BSOFT,831.70,2.00,-23.57,97.27,50.79,104573.0,-33145.0,107508.0,-1.82,48.0,H-SC,10.28,171.0,-0.32,0.76,25.17,XR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-1.87,-19.43,62.92,31.26,116981.0,-44848.0,185920.0,-62.40,30.0,X-MC,20.03,56.0,-0.38,1.32,0.00,X40N,ATH,IT
45,JIOFIN,387.00,-1.13,-8.33,35.79,24.48,87721.0,-22265.0,245100.0,-9.86,36.0,X-LC,1.63,37.0,-0.25,1.74,42.07,XY24,BTT,FINANCE
36,ICICIPRULI,790.00,-1.03,11.49,17.25,30.73,37888.0,22637.0,219642.0,-14.61,60.0,X-MC,6.98,79.0,0.60,1.56,25.73,X40,ATH,INSURANCE
31,HAVELLS,2069.16,-0.98,-8.05,44.07,32.48,132916.0,-26389.0,301602.0,-12.64,48.0,X-MC,5.04,69.0,-0.20,2.14,3.33,X40,ATH,ELECTRICAL
59,RELIANCE,1952.00,-0.98,-5.72,32.91,25.30,62352.0,-11501.0,189462.0,-8.81,34.0,X-LC,7.42,26.0,-0.18,1.34,26.91,XY25,BTT,REFINERIES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,-0.4,-0.11,20.87,20.74,42356.0,-220.0,202950.0,31.15,55.0,M-MC,5.47,189.0,-0.01,1.44,10.5,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.71,-3.48,115.40,107.91,164460.0,-5131.0,142513.0,-21.36,48.0,M-SC,10.17,220.0,-0.03,1.01,6.26,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.02,-14.91,17.55,0.02,26066.0,-26032.0,148527.0,126.56,52.0,M-SC,8.57,240.0,-1.00,1.05,29.19,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,1.64,-16.60,58.30,32.03,64103.0,-21882.0,109953.0,-9.82,67.0,H-SC,6.69,157.0,-0.34,0.78,23.16,OX40N,NTT,IT
77,TTKPRESTIG,770.00,0.41,-22.01,28.33,0.09,22267.0,-22177.0,78600.0,82.50,36.0,M-SC,3.45,253.0,-1.00,0.56,3.26,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.88,-22.34,46.12,13.47,96581.0,-60255.0,209412.0,-68.12,48.0,H-SC,4.35,159.0,-0.62,1.49,6.48,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,-0.23,6.06,12.41,19.23,19804.0,9120.0,159581.0,24.27,71.0,L-LC,8.34,182.0,0.46,1.13,28.63,XY25,ATH,MINING
84,WIPRO,420.00,0.38,8.32,59.03,72.27,108350.0,14102.0,183550.0,-6.99,52.0,M-LC,6.29,101.0,0.13,1.30,15.01,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.38,8.32,59.03,72.27,108350.0,14102.0,183550.0,-6.99,52.0,M-LC,6.29,101.0,0.13,1.30,15.01,XR,NTT,IT
25,FINCABLES,1641.55,0.71,-3.48,115.40,107.91,164460.0,-5131.0,142513.0,-21.36,48.0,M-SC,10.17,220.0,-0.03,1.01,6.26,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.28,-7.19,122.73,106.72,140488.0,-8867.0,114469.0,-54.06,38.0,H-SC,7.80,138.0,-0.06,0.81,14.51,AR,ATH,MISC
37,IEX,219.00,-0.30,-3.90,54.66,48.63,105030.0,-7803.0,192151.0,-36.00,51.0,H-SC,13.18,137.0,-0.07,1.36,7.26,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,0.09,-5.53,43.57,35.63,86261.0,-11591.0,197983.0,-18.95,36.0,H-MC,3.43,119.0,-0.13,1.40,10.22,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.06,-17.11,33.65,10.78,67031.0,-41118.0,199200.0,-50.55,14.0,X-LC,7.86,1.0,-0.61,1.41,0.31,X40,NTT,FMCG
7,AWL,485.00,1.09,-27.98,118.25,57.19,277754.0,-91240.0,234887.0,-65.93,26.0,X-MC,9.95,58.0,-0.33,1.67,1.09,XY24,NTT,FMCG
22,DIXON,18931.72,-1.87,-19.43,62.92,31.26,116981.0,-44848.0,185920.0,-62.40,30.0,X-MC,20.03,56.0,-0.38,1.32,0.00,X40N,ATH,IT
11,BATAINDIA,2096.00,0.48,-41.03,129.27,35.20,98089.0,-52791.0,75879.0,-0.51,30.0,X-SC,17.48,93.0,-0.54,0.54,0.48,X40,NTT,FOOTWEAR
52,PAGEIND,51605.08,0.52,-12.56,49.39,30.62,102371.0,-29784.0,207270.0,-37.83,30.0,X-MC,16.83,57.0,-0.29,1.47,0.77,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,-0.02,-2.50,23.86,20.77,54471.0,-5848.0,228296.0,-10.93,30.0,X-MC,1.57,70.0,-0.11,1.62,4.25,X40N,NTT,BREWERIES
45,JIOFIN,387.00,-1.13,-8.33,35.79,24.48,87721.0,-22265.0,245100.0,-9.86,36.0,X-LC,1.63,37.0,-0.25,1.74,42.07,XY24,BTT,FINANCE
65,SIEMENS,4671.50,0.88,-19.21,55.35,25.51,83219.0,-35745.0,150350.0,-7.73,40.0,H-LC,2.12,50.0,-0.43,1.07,10.80,AR,ATH,ELECTRICAL
20,DABUR,735.00,-0.73,1.70,41.29,43.70,104173.0,4229.0,252297.0,-6.68,62.0,X-MC,2.16,73.0,0.04,1.79,17.01,XY24,BTT,FMCG
73,TCS,4389.97,0.74,-10.27,34.52,20.71,117162.0,-38833.0,339404.0,-22.65,54.0,X-LC,2.28,3.0,-0.33,2.41,13.41,X40,ATH,IT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-1.87,-19.43,62.92,31.26,116981.0,-44848.0,185920.0,-62.40,30.0,X-MC,20.03,56.0,-0.38,1.32,0.00,X40N,ATH,IT
43,ITC,452.00,-0.06,-17.11,33.65,10.78,67031.0,-41118.0,199200.0,-50.55,14.0,X-LC,7.86,1.0,-0.61,1.41,0.31,X40,NTT,FMCG
3,ACC,3906.00,0.14,-28.17,128.54,64.15,219688.0,-67041.0,170910.0,-58.06,35.0,X-SC,6.49,84.0,-0.31,1.21,0.35,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,0.48,-41.03,129.27,35.20,98089.0,-52791.0,75879.0,-0.51,30.0,X-SC,17.48,93.0,-0.54,0.54,0.48,X40,NTT,FOOTWEAR
58,RELAXO,1176.00,0.63,-48.67,196.74,52.31,146579.0,-70656.0,74504.0,-45.31,39.0,X-SC,7.82,92.0,-0.48,0.53,0.63,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,0.15,-28.53,99.87,42.84,46399.0,-18547.0,46459.0,-53.17,48.0,X-SC,25.97,83.0,-0.40,0.33,5.11,XY24,NTT,MISC
58,RELAXO,1176.00,0.63,-48.67,196.74,52.31,146579.0,-70656.0,74504.0,-45.31,39.0,X-SC,7.82,92.0,-0.48,0.53,0.63,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.48,-41.03,129.27,35.20,98089.0,-52791.0,75879.0,-0.51,30.0,X-SC,17.48,93.0,-0.54,0.54,0.48,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,0.61,0.29,23.72,24.08,30200.0,367.0,127317.0,-7.89,51.0,X-SC,5.89,91.0,0.01,0.90,19.96,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.57,-18.08,74.96,43.32,100012.0,-29452.0,133420.0,-29.66,43.0,X-SC,8.07,90.0,-0.29,0.95,2.44,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.06,-17.11,33.65,10.78,67031.0,-41118.0,199200.0,-50.55,14.0,X-LC,7.86,1.0,-0.61,1.41,0.31,X40,NTT,FMCG
75,TMPV,600.00,-0.56,-28.55,72.12,22.98,116403.0,-64496.0,161402.0,-25.52,37.0,X-LC,3.82,2.0,-0.55,1.15,1.51,XY24,NTT,AUTO
73,TCS,4389.97,0.74,-10.27,34.52,20.71,117162.0,-38833.0,339404.0,-22.65,54.0,X-LC,2.28,3.0,-0.33,2.41,13.41,X40,ATH,IT
81,VBL,671.64,0.72,0.46,35.04,35.67,111185.0,1467.0,317309.0,-11.55,58.0,X-LC,2.39,5.0,0.01,2.25,14.38,X40N,ATH,FMCG
41,INFY,1972.00,0.14,8.80,23.39,34.24,79624.0,27529.0,340417.0,-15.29,46.0,X-LC,7.58,6.0,0.35,2.42,17.81,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.11,-33.23,99.02,32.89,51049.0,-25652.0,51554.0,-736.93,64.0,L-MC,8.15,259.0,-0.50,0.37,41.97,XR,NTT,BANKS
13,BSOFT,831.70,2.00,-23.57,97.27,50.79,104573.0,-33145.0,107508.0,-1.82,48.0,H-SC,10.28,171.0,-0.32,0.76,25.17,XR,ATH,IT
69,SURYODAY,216.00,-0.21,-18.67,51.27,23.03,74673.0,-33425.0,145646.0,57.89,54.0,H-SC,4.13,165.0,-0.45,1.03,44.51,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.02,-14.91,17.55,0.02,26066.0,-26032.0,148527.0,126.56,52.0,M-SC,8.57,240.0,-1.00,1.05,29.19,OX40N,NTT,PAINTS
17,COALINDIA,484.83,-0.23,6.06,12.41,19.23,19804.0,9120.0,159581.0,24.27,71.0,L-LC,8.34,182.0,0.46,1.13,28.63,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-0.85,-30.33,62.04,12.90,94447.0,-66261.0,152235.0,124.76,59.0,H-SC,9.69,173.0,-0.70,1.08,26.58,OX40N,NTT,FINANCE
72,TATAELXSI,9161.00,1.64,-16.60,58.30,32.03,64103.0,-21882.0,109953.0,-9.82,67.0,H-SC,6.69,157.0,-0.34,0.78,23.16,OX40N,NTT,IT
17,COALINDIA,484.83,-0.23,6.06,12.41,19.23,19804.0,9120.0,159581.0,24.27,71.0,L-LC,8.34,182.0,0.46,1.13,28.63,XY25,ATH,MINING
57,RECLTD,446.00,-0.40,-0.11,20.87,20.74,42356.0,-220.0,202950.0,31.15,55.0,M-MC,5.47,189.0,-0.01,1.44,10.50,XY25,NTT,FINANCE
40,INDUSINDBK,1800.00,-0.11,-33.23,99.02,32.89,51049.0,-25652.0,51554.0,-736.93,64.0,L-MC,8.15,259.0,-0.50,0.37,41.97,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.37
1,25,44.42
2,50,76.48


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.57
MC    30.71
LC    26.68
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.22
X40      24.02
X40N     12.77
XY25     10.85
XR        8.97
AR        8.54
OX40N     7.63
SR        0.96
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.32
X-LC    23.18
H-SC    22.77
M-SC    11.20
X-SC     7.83
H-MC     4.58
M-MC     1.44
M-LC     1.30
L-LC     1.13
H-LC     1.07
L-SC     0.77
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.26,-8.20,43.70
IT,12.90,-17.57,76.88
FINANCE,10.99,-15.10,63.42
MISC,7.02,-31.11,84.74
ELECTRICAL,5.87,-13.51,55.09
PAINTS,5.82,-12.65,29.38
INSURANCE,4.70,-2.35,37.37
PHARMA,3.96,-4.94,37.98
AUTO,2.77,-34.16,80.68


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3274131.0,21
AR,1367347.0,10
XR,1227374.0,12
X40,1135051.0,15
X40N,942047.0,9
OX40N,714041.0,10
XY25,411041.0,7
SR,285840.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3584240.0,24
X-MC,1604362.0,16
M-SC,1477787.0,15
X-LC,955953.0,13
X-SC,816163.0,8
H-MC,431855.0,3
L-SC,181734.0,2
M-LC,108350.0,1
H-LC,83219.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1270007.0      6
           AR         926982.0      5
M-SC       XY24       847404.0      6
H-SC       XR         790890.0      7
X-MC       X40        523422.0      7
           XY24       450088.0      3
X-LC       X40        418379.0      6
X-MC       X40N       409701.0      4
X-SC       X40N       326626.0      3
H-SC       OX40N      310521.0      4
M-SC       OX40N      310453.0      5
X-SC       XY24       296287.0      3
H-SC       SR         285840.0      2
H-MC       AR         225634.0      2
X-MC       XY25       221151.0      2
H-MC       XY24       206221.0      1
X-LC       X40N       205720.0      2
           XY24       204124.0      2
X-SC       X40        193250.0      2
M-SC       XR         188418.0      2
           AR         131512.0      2
X-LC       XY25       127730.0      3
M-LC       XR         108350.0      1
L-SC       OX40N       93067.0      1
           XR          88667.0      1
H-LC       AR          83219.0      1
L-MC       XR          51049.0      1
M-MC       XY25        42356.0      1
L-LC       XY25        19804.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 37.0 seconds
